In [0]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, GlobalMaxPooling1D, Embedding, Conv1D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
dataset = [


    'This movie is excellent',
    'I loved the movie, it was fantastic',
    'The film is brilliant, you should watch',
    'Wonderful movie',
    'one of the best films ever',
    'fantastic film to watch',
    'great movie',
    'Acting and direction is brilliant',



    "poor acting",
    'horrible film',
    'pathetic acting',
    'The film is very boring',
    'I wasted my money',
    'I did not like the film',
    'not recommended',
    'it was a poor story'
]



In [0]:
labels = np.array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [0]:
tokenizer = Tokenizer()
tokenizer .fit_on_texts(dataset)

In [0]:
vocabulary_length = len(tokenizer.word_index) + 1

In [7]:
integer_sentences = tokenizer.texts_to_sequences(dataset)
print(integer_sentences)

[[14, 3, 4, 15], [5, 16, 1, 3, 7, 8, 9], [1, 2, 4, 10, 17, 18, 11], [19, 3], [20, 21, 1, 22, 23, 24], [9, 2, 25, 11], [26, 3], [6, 27, 28, 4, 10], [12, 6], [29, 2], [30, 6], [1, 2, 4, 31, 32], [5, 33, 34, 35], [5, 36, 13, 37, 1, 2], [13, 38], [7, 8, 39, 12, 40]]


In [8]:
from nltk.tokenize import word_tokenize

token_count = lambda sentence: len(word_tokenize(sentence))
max_sentence = max(dataset, key = token_count)
max_sentence_length = len(word_tokenize(max_sentence))

padded_reviews = pad_sequences(integer_sentences , max_sentence_length, padding='post')

print(padded_reviews)

[[14  3  4 15  0  0  0  0]
 [ 5 16  1  3  7  8  9  0]
 [ 1  2  4 10 17 18 11  0]
 [19  3  0  0  0  0  0  0]
 [20 21  1 22 23 24  0  0]
 [ 9  2 25 11  0  0  0  0]
 [26  3  0  0  0  0  0  0]
 [ 6 27 28  4 10  0  0  0]
 [12  6  0  0  0  0  0  0]
 [29  2  0  0  0  0  0  0]
 [30  6  0  0  0  0  0  0]
 [ 1  2  4 31 32  0  0  0]
 [ 5 33 34 35  0  0  0  0]
 [ 5 36 13 37  1  2  0  0]
 [13 38  0  0  0  0  0  0]
 [ 7  8 39 12 40  0  0  0]]


In [9]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros

embedd_dict= dict()
glove_embeddings = open('/gdrive/My Drive/datasets/glove.6B.100d.txt', encoding="utf8")


In [0]:
for embeddings in glove_embeddings:
    embedding_tokens = embeddings.split()
    emb_word = embedding_tokens [0]
    emb_vector = asarray(   embedding_tokens[1:], dtype='float32')
    embedd_dict [emb_word] = emb_vector 

glove_embeddings.close()

In [0]:
embedd_mat= zeros((vocabulary_length, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embedd_dict.get(word)
    if embedding_vector is not None:
        embedd_mat[index] = embedding_vector

In [13]:
embedd_mat.shape

(41, 100)

In [0]:
embedding_inputs = Input(shape=(max_sentence_length))
embedding_layer = Embedding(vocabulary_length, 100, weights=[embedd_mat], trainable=False)(embedding_inputs)
flatten_layer = Flatten()(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(flatten_layer)
model = Model(inputs=embedding_inputs, outputs=output_layer)

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
embedding (Embedding)        (None, 8, 100)            4100      
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 801       
Total params: 4,901
Trainable params: 801
Non-trainable params: 4,100
_________________________________________________________________
None


In [16]:
model.fit(padded_reviews, labels, epochs=100, verbose=1)
loss, accuracy = model.evaluate(padded_reviews, labels, verbose=0)

print('Accuracy: %f' % (accuracy*100))

Epoch 1/100
1/1 [==============================] - 0s 2ms/step - loss: 0.9458 - acc: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 0.8986 - acc: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 0.8539 - acc: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 0.8119 - acc: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 0.7729 - acc: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.7370 - acc: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 2ms/step - loss: 0.7042 - acc: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6746 - acc: 0.5625
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6481 - acc: 0.6250
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6246 - acc: 0.6250
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6037 